In [1]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
path = 'text.csv'

In [3]:
df = pd.read_csv('text.csv')

In [4]:
df

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...,...
416804,416804,i feel like telling these horny devils to find...,2
416805,416805,i began to realize that when i was feeling agi...,3
416806,416806,i feel very curious be why previous early dawn...,5
416807,416807,i feel that becuase of the tyranical nature of...,3


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [6]:
X = df['text']
y = df['label']

In [7]:
print(len(X))

416809


In [8]:
def wp(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [9]:
X

0             i just feel really helpless and heavy hearted
1         ive enjoyed being able to slouch about relax a...
2         i gave up my internship with the dmrg and am f...
3                                i dont know i feel so lost
4         i am a kindergarten teacher and i am thoroughl...
                                ...                        
416804    i feel like telling these horny devils to find...
416805    i began to realize that when i was feeling agi...
416806    i feel very curious be why previous early dawn...
416807    i feel that becuase of the tyranical nature of...
416808    i think that after i had spent some time inves...
Name: text, Length: 416809, dtype: object

In [10]:
X = X.apply(wp)
X

0             i just feel really helpless and heavy hearted
1         ive enjoyed being able to slouch about relax a...
2         i gave up my internship with the dmrg and am f...
3                                i dont know i feel so lost
4         i am a kindergarten teacher and i am thoroughl...
                                ...                        
416804    i feel like telling these horny devils to find...
416805    i began to realize that when i was feeling agi...
416806    i feel very curious be why previous early dawn...
416807    i feel that becuase of the tyranical nature of...
416808    i think that after i had spent some time inves...
Name: text, Length: 416809, dtype: object

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [25]:
#Конвертуємо текст в вектор

vectorization = TfidfVectorizer()
XV_train = vectorization.fit_transform(X_train)
XV_test = vectorization.transform(X_test)

In [26]:
XV_test 

<83362x67923 sparse matrix of type '<class 'numpy.float64'>'
	with 1293065 stored elements in Compressed Sparse Row format>

In [13]:
score_baseline = y.value_counts(normalize=True).max()
score_baseline

0.3384451871240784

In [14]:
params = {
    'objective': 'binary:logistic',
    'n_estimators': 500,
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_jobs': -1,
    'tree_method': 'hist',    
}

In [15]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(**params)

In [16]:
xgb_model.fit(XV_train, y_train, eval_set=[(XV_test, y_test)],verbose=50)

[0]	validation_0-mlogloss:1.73901
[50]	validation_0-mlogloss:0.96609
[100]	validation_0-mlogloss:0.74568
[150]	validation_0-mlogloss:0.61939
[200]	validation_0-mlogloss:0.53522
[250]	validation_0-mlogloss:0.47430
[300]	validation_0-mlogloss:0.42796
[350]	validation_0-mlogloss:0.39203
[400]	validation_0-mlogloss:0.36292
[450]	validation_0-mlogloss:0.33917
[499]	validation_0-mlogloss:0.32026


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=-1,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [17]:
pred_xgb=xgb_model.predict(XV_test)

In [18]:
from sklearn.metrics import accuracy_score
score_xgb = accuracy_score(y_test, pred_xgb)

In [19]:
score_xgb

0.8979990883136201

In [20]:
def output_lable(n):
    return f'The Text Sentement is {classes_names[n]}'

In [21]:
classes_names = { 0:'sadness',1:'joy',2:'love',3:'anger',4:'fear',5:'surprise'}
classes_names

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [22]:
def output_lable(n):
    return f'The Text Sentement is {classes_names[n]}'
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wp) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_xgb = xgb_model.predict(new_xv_test)
    print(f'{output_lable(pred_xgb[0])}')

In [23]:
text = "i dont know i feel so lost"
manual_testing(text)

The Text Sentement is sadness


In [24]:
text = 'i didn t feel terrific'
manual_testing(text)

The Text Sentement is joy


In [40]:
import pickle
with open('my_model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)

In [47]:
text = 'shit movie'

manual_testing(text)

The Text Sentement is joy


In [48]:
xgb_model.save_model('my_model.xgb')

C:\ProgramData\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [02:19:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
